In [50]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np

Le but de ce model est de prédire à partir d'autres stock une certaine stock afin de définir une potentielle anomalie niveau comportement par rapport aux autres stock. Nous avons choisi d'utiliser keras qui est l'API de tensorflow et plus user friendly surtout dans l'initialisation de l'architecture.

In [19]:
# Import data
data = pd.read_csv('data_stocks.csv')
# Drop date variable
data = data.drop(['DATE'], 1)
# Dimensions of dataset
n = data.shape[0]
p = data.shape[1]
# Make data a numpy array
df = data
data = data.values

On sépare notre base de donnée en base de données train pour faire l'entrainement et une base de données test afin d'éviter de l'overfitting. 80% de notre data est pris pour la base de données test alors que les 20% finaux sont conservées pour la bdd test

In [20]:
# Training and test data
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end
test_end = n
data_train = data[np.arange(train_start, train_end), :]
data_test = data[np.arange(test_start, test_end), :]


Afin de set nos données nous avons scale les informations pour les initializer et scale les données test en fonction de ces données de train

In [21]:
# Scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_train = scaler.fit_transform(data_train)
data_test = scaler.transform(data_test)
# Build X and y
X_train = data_train[:, 1:]
y_train = data_train[:, 0]
X_test = data_test[:, 1:]
y_test = data_test[:, 0]

Notre réseau de données a une architecture de 4 hidden layers (1024,512,256,126) avec comme fonction d'activation 'relu' avec un dropout de 0.1 qui réduit l'overfitting et nous renvoie un mse diminué par rapport à un model sans dropout.

In [55]:
model = Sequential()
model.add(Dense(1024, input_dim=500, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='relu'))

Comme la variable cible est une variable continue et on cherche une relation linéaire, nous avons décidé de prendre comme loss MSE avec comme optimizer ADAM afin d'avoir un gradient déscendant ni trop lent ni trop rapide et ne pas être coincé dans un point selle

In [56]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

On fait fit notre model avec un propagation et une retropropagation avec 10 itération de toute la données sur des batches de tailles 256 afin de réaliser notre 

In [57]:
model.fit(X_train, y_train, epochs=10, batch_size=256)

Epoch 1/10
33012/33012 [==============================] - 4s 112us/step - loss: 0.0837 - mse: 0.0837
Epoch 2/10
33012/33012 [==============================] - 4s 108us/step - loss: 9.1108e-04 - mse: 9.1108e-04
Epoch 3/10
33012/33012 [==============================] - 3s 106us/step - loss: 6.8883e-04 - mse: 6.8883e-04
Epoch 4/10
33012/33012 [==============================] - 4s 108us/step - loss: 5.2977e-04 - mse: 5.2977e-04
Epoch 5/10
33012/33012 [==============================] - 3s 106us/step - loss: 4.9124e-04 - mse: 4.9124e-04
Epoch 6/10
33012/33012 [==============================] - 3s 106us/step - loss: 4.7452e-04 - mse: 4.7452e-04
Epoch 7/10
33012/33012 [==============================] - 4s 109us/step - loss: 3.7035e-04 - mse: 3.7035e-04
Epoch 8/10
33012/33012 [==============================] - 4s 106us/step - loss: 3.8600e-04 - mse: 3.8600e-04
Epoch 9/10
33012/33012 [==============================] - 4s 113us/step - loss: 3.4426e-04 - mse: 3.4425e-04
Epoch 10/10
33012/33012 [==

In [58]:
_, accuracy = model.evaluate(X_test, y_test)
print('MSE: %.6f' % (accuracy))

8254/8254 [==============================] - 1s 86us/step
MSE: 0.003234


501